In [4]:
# This block allows us to import from the benchmark folder,
# as if it was a package installed using pip
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
import torch
from attrbench import datasets, attribution, models
from attrbench.evaluation.impact_coverage import impact_coverage
from os import path
import itertools

In [6]:
device = "cpu"
dataset_name = "CIFAR10"
model_name = "resnet18"
download_dataset = False
batch_size = 8
n_batches = 32
target_label = 0
data_root = "../data"
normalize_attrs = True
aggregation_fn = "avg"

In [8]:
patch = torch.load(path.join(data_root, "patches", f"{dataset_name}_{model_name}_patch.pt"))
dataset = datasets.Cifar(batch_size=batch_size, data_location=path.join(data_root, dataset_name))
model = models.Resnet(version=model_name, params_loc=path.join(data_root, f"models/{dataset_name}/{model_name}.pt"),
                      output_logits=True, num_classes=10)
model.to(device)
model.eval()
pass  # Prevents output from model.eval()

In [11]:
kwargs = {
    "normalize": normalize_attrs,
    "aggregation_fn": aggregation_fn
}

attribution_methods = {
    # "GuidedGradCAM": attribution.GuidedGradCAM(model, model.get_last_conv_layer(), **kwargs),
    "Gradient": attribution.Gradient(model, **kwargs),
    "InputXGradient": attribution.InputXGradient(model, **kwargs),
    "IntegratedGradients": attribution.IntegratedGradients(model, **kwargs),
    "GuidedBackprop": attribution.GuidedBackprop(model, **kwargs),
    "Deconvolution": attribution.Deconvolution(model, **kwargs),
    "Random": attribution.Random(**kwargs)
}

In [12]:
impact_coverage(data=itertools.islice(dataset.get_dataloader(train=False), n_batches),
                sample_shape=dataset.sample_shape, patch=patch, model=model, methods=attribution_methods,
                device=device, target_label=0)

{'Gradient': 0.08843537414965986,
 'InputXGradient': 0.09717868338557993,
 'IntegratedGradients': 0.10402838930143918,
 'GuidedBackprop': 0.08059432687978388,
 'Deconvolution': 0.07464977931299174,
 'Random': 0.05177487009328242}

In [5]:
torch.cuda.is_available()

False